# Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import time
import pandas    as pd
import numpy     as np
from   tqdm      import tqdm


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Visualization library
#
import matplotlib.pyplot   as plt 



# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# User libraries
#
from utils.non_parametric_tests import *

# Statistical analysis

## Friedman Aligned Rankings

In [19]:
df = pd.DataFrame( [] )

df['Method1'] = np.random.rand( 100 ) 
df['Method2'] = np.random.rand( 100 ) 
df['Method3'] = np.random.rand( 100 ) 

df.head(3)

,Method1,Method2,Method3
0,0.783503,0.320871,0.126619
1,0.756844,0.050731,0.414192
2,0.069546,0.422955,0.091816


In [23]:
T, p_value, rankings_avg, rankings_cmp = friedman_aligned_ranks_test( df )

d = {}
for i, feature in enumerate(df.columns):
    d[ feature ] = rankings_avg[i]

[INFO] Number of methods:   3
[INFO] Number of problems:  100


In [26]:
Ranking = pd.DataFrame( [] )
Ranking['Methods'] = df.columns
Ranking['FAR']     = rankings_avg

Ranking.sort_values(by = 'FAR')

,Methods,FAR
1,Method2,142.63
2,Method3,145.19
0,Method1,163.68


## Finner post-hoc test

In [27]:
comparisons, _, _, adj_p_values = finner_test( d )

Finner = pd.DataFrame( [] )
Finner['Comparisons']     = comparisons
Finner['APV']             = adj_p_values
Finner['Null hypothesis'] = Finner['APV'].apply(lambda x: 'Rejected' if x > 0.05 else 'Failed to reject')

Finner

[INFO] Control method:  Method2


,Comparisons,APV,Null hypothesis
0,Method2 vs Method1,0.000000,Failed to reject
1,Method2 vs Method3,0.010467,Failed to reject
